TASK : Restaurent Recommentation

Objective: Create a restaurant recommendation
system based on user preferences.

Steps:

Preprocess the dataset by handling missing
values and encoding categorical variables.
Determine the criteria for restaurant
recommendations (e.g., cuisine preference,
price range).
Implement a content-based filtering
approach where users are recommended
restaurants similar to their preferred criteria.
Test the recommendation system by
providing sample user preferences and
evaluating the quality of recommendations.

In [8]:
import pandas as pd
import os

print(os.getcwd())

d:\Python\AI & ML\Cognify


In [9]:
df_rec = pd.read_csv('Dataset .csv')
df_rec.head()

Restaurant ID         Restaurant Name  Country Code              City  \
0        6317637        Le Petit Souffle           162       Makati City   
1        6304287        Izakaya Kikufuji           162       Makati City   
2        6300002  Heat - Edsa Shangri-La           162  Mandaluyong City   
3        6318506                    Ooma           162  Mandaluyong City   
4        6314302             Sambo Kojin           162  Mandaluyong City   

                                             Address  \
0  Third Floor, Century City Mall, Kalayaan Avenu...   
1  Little Tokyo, 2277 Chino Roces Avenue, Legaspi...   
2  Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...   
3  Third Floor, Mega Fashion Hall, SM Megamall, O...   
4  Third Floor, Mega Atrium, SM Megamall, Ortigas...   

                                     Locality  \
0   Century City Mall, Poblacion, Makati City   
1  Little Tokyo, Legaspi Village, Makati City   
2  Edsa Shangri-La, Ortigas, Mandaluyong City   
3      SM Megamall, Ortigas, Mandaluyong City   
4      SM Megamall, Ortigas, Mandaluyong City   

                                    Locality Verbose   Longitude   Latitude  \
0  Century City Mall, Poblacion, Makati City, Mak...  121.027535  14.565443   
1  Little Tokyo, Legaspi Village, Makati City, Ma...  121.014101  14.553708   
2  Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...  121.056831  14.581404   
3  SM Megamall, Ortigas, Mandaluyong City, Mandal...  121.056475  14.585318   
4  SM Megamall, Ortigas, Mandaluyong City, Mandal...  121.057508  14.584450   

                           Cuisines  ...          Currency Has Table booking  \
0        French, Japanese, Desserts  ...  Botswana Pula(P)               Yes   
1                          Japanese  ...  Botswana Pula(P)               Yes   
2  Seafood, Asian, Filipino, Indian  ...  Botswana Pula(P)               Yes   
3                   Japanese, Sushi  ...  Botswana Pula(P)                No   
4                  Japanese, Korean  ...  Botswana Pula(P)               Yes   

  Has Online delivery Is delivering now Switch to order menu Price range  \
0                  No                No                   No           3   
1                  No                No                   No           3   
2                  No                No                   No           4   
3                  No                No                   No           4   
4                  No                No                   No           4   

   Aggregate rating  Rating color Rating text Votes  
0               4.8    Dark Green   Excellent   314  
1               4.5    Dark Green   Excellent   591  
2               4.4         Green   Very Good   270  
3               4.9    Dark Green   Excellent   365  
4               4.8    Dark Green   Excellent   229  

[5 rows x 21 columns]

Preprocess the Dataset :

--  Drop unnecessary columns

--  Handle missing values

--  Keep useful features like: Restaurant Name, Cuisines, Average Cost for two, Price range, Aggregate rating

In [10]:
# Select relevant columns
df_rec = df[['Restaurant Name', 'Cuisines', 'Average Cost for two', 'Price range', 'Aggregate rating']].copy()

# Fill missing cuisine values
df_rec['Cuisines'].fillna(df_rec['Cuisines'].mode()[0], inplace=True)

# Drop duplicates
df_rec.drop_duplicates(inplace=True)

df_rec.head()


C:\Users\manit\AppData\Local\Temp\ipykernel_31360\2978527301.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_rec['Cuisines'].fillna(df_rec['Cuisines'].mode()[0], inplace=True)


Restaurant Name                          Cuisines  \
0        Le Petit Souffle        French, Japanese, Desserts   
1        Izakaya Kikufuji                          Japanese   
2  Heat - Edsa Shangri-La  Seafood, Asian, Filipino, Indian   
3                    Ooma                   Japanese, Sushi   
4             Sambo Kojin                  Japanese, Korean   

   Average Cost for two  Price range  Aggregate rating  
0                  1100            3               4.8  
1                  1200            3               4.5  
2                  4000            4               4.4  
3                  1500            4               4.9  
4                  1500            4               4.8

Convert Cuisines to Feature Vectors (Text to Numbers)

In [11]:
#-- Use TF-IDF vectorization for textual data in the Cuisines column.

from sklearn.feature_extraction.text import TfidfVectorizer

'''
- This tool converts a collection of raw text into a matrix of TF-IDF scores, which helps algorithms understand the relative importance of words.
- you're about to apply TF-IDF vectorization to text data in the "Cuisines" column.
'''

# Vectorize cuisine text
tfidf = TfidfVectorizer(stop_words='english')

'''
- creating a TfidfVectorizer object.
- stop_words='english' means it will ignore common words like "and", "the", "in" which don’t add meaningful insight.
- The vectorizer will compute scores for unique words in the "Cuisines" text.
'''
cuisine_matrix = tfidf.fit_transform(df_rec['Cuisines'])

'''
- ⚙️ This step does two things:
- fit: Learns all unique words across the "Cuisines" column.
- transform: Converts each row of text into a numeric vector based on word importance.
- The result is a sparse matrix (cuisine_matrix) where:
- Each row represents a restaurant.
- Each column represents a unique cuisine-related word.
- Each cell contains the TF-IDF score, which reflects how relevant a word is to a specific restaurant, while penalizing common words
'''

'\n- ⚙️ This step does two things:\n- fit: Learns all unique words across the "Cuisines" column.\n- transform: Converts each row of text into a numeric vector based on word importance.\n- The result is a sparse matrix (cuisine_matrix) where:\n- Each row represents a restaurant.\n- Each column represents a unique cuisine-related word.\n- Each cell contains the TF-IDF score, which reflects how relevant a word is to a specific restaurant, while penalizing common words\n'

Compute Similarity Based on Cuisines

In [12]:
#-- Use cosine similarity to compare user preferences with restaurants.

from sklearn.metrics.pairwise import cosine_similarity #- This is used for comparing vector data, especially in NLP and recommendation systems.

# Compute similarity matrix
similarity_matrix = cosine_similarity(cuisine_matrix)

'''  
- You apply cosine similarity to the TF-IDF matrix (cuisine_matrix) where:
- Each row represents one restaurant’s cuisine.
- Each column represents a unique cuisine-related word.
- The result is a square matrix (similarity_matrix) of size n x n (where n is number of restaurants), containing scores from 0 to 1:
- 1.0 = perfect match (same cuisine description)
- Closer to 0 = very different cuisine descriptions
'''


'  \n- You apply cosine similarity to the TF-IDF matrix (cuisine_matrix) where:\n- Each row represents one restaurant’s cuisine.\n- Each column represents a unique cuisine-related word.\n- The result is a square matrix (similarity_matrix) of size n x n (where n is number of restaurants), containing scores from 0 to 1:\n- 1.0 = perfect match (same cuisine description)\n- Closer to 0 = very different cuisine descriptions\n'

Build the Recommendation Function


In [ ]:
def recommend_restaurants(user_cuisine, price_range, top_n=5): 
    # Vectorize user cuisine input
    user_vector = tfidf.transform([user_cuisine])   
    ''' 
    - Converts the user's cuisine preference into a TF-IDF vector.
    - This vector is now compatible with your restaurant dataset’s TF-IDF vectors for comparison.
    '''
    # Compute similarity with all restaurants
    similarity_scores = cosine_similarity(user_vector, cuisine_matrix).flatten()

    ''' 
    - Compares the user’s vector to each restaurant’s cuisine vector using cosine similarity.
    - .flatten() converts the resulting matrix into a 1D array where:
    - Each value represents how similar a restaurant is to the user’s input.

    '''
    
    # Add scores to dataframe
    df_rec['similarity'] = similarity_scores

    ''' 
    - Adds a new column named 'similarity' to your df_rec DataFrame.
    - Each restaurant now has a similarity score based on its cuisine match to the user.
    '''

    # Filter by price range
    filtered_df = df_rec[df_rec['Price range'] == price_range]
    
    ''' 
    - Filters the DataFrame to include only restaurants that match the user’s specified price range.
    '''
    # Sort by similarity and rating
    recommendations = filtered_df.sort_values(by=['similarity', 'Aggregate rating'], ascending=[False, False])

    ''' 
    - Sorts the filtered results by:
    - similarity (most relevant cuisines first)
    - Aggregate rating (highest rated restaurants first)
    - Uses ascending=False to sort in descending order.

    '''
    
    return recommendations[['Restaurant Name', 'Cuisines', 'Average Cost for two', 'Aggregate rating']].head(top_n)


Test with Sample User Input

In [14]:
# Example: User likes 'Italian, Pizza' and prefers mid-range price (3)
recommend_restaurants(user_cuisine='Italian, Pizza', price_range=3)


Restaurant Name        Cuisines  Average Cost for two  \
255                         Centro  Italian, Pizza                    40   
566                 Pizza Di Rocco  Italian, Pizza                   150   
166   Biaggi's Ristorante Italiano  Italian, Pizza                    40   
16        Nonna's Pasta & Pizzeria  Italian, Pizza                   850   
8062                  PizzaExpress  Italian, Pizza                  1600   

      Aggregate rating  
255                4.5  
566                4.4  
166                4.1  
16                 4.0  
8062               3.9